# Novelty

In [5]:
from ipynb.fs.full.koselleck import *

ModuleNotFoundError: No module named 'ipynb.fs.full.koselleck'

In [3]:
def nov_word(word,progress=False,**kwargs):
    odf=test_novelty(get_historical_semantic_distance_matrix(
            word,
            interpolate=True,
            normalize=True,
            progress=progress
        ),
        **kwargs
    )
    if odf is not None and len(odf):
        odf=odf.query('foote_novelty!=0').assign(word=word)
    return odf

In [4]:
nov_word('virtue')

NameError: name 'test_novelty' is not defined

In [2]:

def get_novelty(words=None,interpolate=True,by_word=True,progress=True, lim=None, num_proc=1, force=False,use_all_word_cache=False, **y):
    if words is None: words=get_valid_words()
    words=tokenize_fast(words) if type(words)==str else words
    veclib=get_veclib()
    if use_all_word_cache and os.path.exists(FN_NOV_ALL_BYWORD):
        odf=get_all_novelty_scores(by_foote_size=True)
        odf=odf[odf.word.isin(words)]
        if not by_word: odf=odf.groupby(['foote_size','period']).mean().reset_index()
    else:
        if by_word:
            o=[]
            words_todo=words
            words_done=pd.DataFrame()
            if not force:
                words_done = pd.DataFrame([
                    dx
                    for word in words
                    for dx in veclib.get(f'nov({word})',[])
                    if type(dx)==dict
                ])
                display(words_done)
                if len(words_done):
                    words_todo = list(set(words) - set(words_done.word))
            
            for dfg in pmap_iter(
                do_get_novelty,
                words_todo[:lim],
                progress=progress,
                desc='Calculating novelty for words',
                num_proc=num_proc,
                kwargs=dict(progress=False)
            ):
                if dfg is None or not len(dfg): continue                
                word=dfg.iloc[0].word
#                 display(dfg)
#                 display(dfg.to_dict('records'))
                veclib[f'nov({word})']=dfg.to_dict('records')
                o+=[dfg]
            odf=words_done.append(pd.concat(o) if len(o) else pd.DataFrame())
        else:
            distdf=get_historical_semantic_distance_matrix(words,interpolate=interpolate)
            odf=test_novelty(distdf,**y)
        odf['period']=[int(x[:4]) for x in odf.period]
    return odf

In [3]:
#do_get_novelty('value')

In [ ]:
all_nov=get_novelty(get_keywords_l(),num_proc=4)

,period,foote_novelty,foote_size,p_peak,p_trough,word
0,1730-1735,3.191489,2,1.00,0.49,consumer
1,1735-1740,-107.402482,2,1.00,0.99,consumer
2,1740-1745,-78.383570,2,1.00,0.99,consumer
3,1745-1750,5.718085,2,1.00,0.26,consumer
4,1750-1755,-24.660165,2,1.00,0.99,consumer
...,...,...,...,...,...,...
2991,1905-1910,635.106383,6,1.00,1.00,reformation
2992,1910-1915,1126.462766,6,0.99,1.00,reformation
2993,1915-1920,1469.015957,6,0.42,1.00,reformation
2994,1920-1925,1663.918440,6,0.11,1.00,reformation


Calculating novelty for words [x4]:   5%|▌         | 11/203 [00:05<02:15,  1.42it/s]Reading from /home/ryan/github/koselleck/data/data.all_models_halfdec.pkl


In [ ]:
all_nov_scores = get_novelty(
    get_valid_words(),
    progress=True,
    num_proc=2
)
all_nov_scores

In [ ]:
def get_all_novelty_scores(ifnfn=FN_NOV_ALL_BYWORD,
                           by_foote_size=False, min_foote_size=6, max_foote_size=6):
    global DFALLNOV
    if DFALLNOV is not None:
        odf=DFALLNOV
    elif not os.path.exists(ifnfn):
        print(f'Generating novelty scores, saving to {ifnfn}')
        DFALLNOV = odf = nov_all_byword = get_novelty(by_word=True,progress=True,num_proc=1)
        nov_all_byword.to_pickle(FN_NOV_ALL_BYWORD)
    else:
        print(f'Loading novelty scores from {ifnfn}')
        with open(ifnfn,'rb') as f: odf=pickle.load(f)
        DFALLNOV=odf
        
    odf = odf.query(f'foote_novelty!=0 & {min_foote_size}<=foote_size<={max_foote_size}')
    odf = pd.concat(
        grp.assign(
            foote_novelty_z=(grp.foote_novelty - grp.foote_novelty.mean()) / grp.foote_novelty.std()
        )
        for i,grp in odf.groupby('foote_size')
    )
    if not by_foote_size:
        odf=odf.groupby(['word','period']).mean().drop('foote_size',1).reset_index()
    else:
        odf['foote_size']=odf.foote_size.apply(int)
        
    #odf=odf.query('period<1900')
    return odf
        


In [ ]:
get_novelty('virtue,value',by_word=True)

In [ ]:
# get_novelty('station', by_word=False).query('foote_novelty!=0')

In [ ]:
# nov_all_mean=get_novelty(by_word=False)
# nov_all_mean

In [ ]:
get_all_novelty_scores(by_foote_size=True)

In [ ]:
def get_signif_novelty_scores(p_peak=0.05,min_peaks=1):
    odf=get_all_novelty_scores().query(f'p_peak<{p_peak}')
    odf=pd.concat(
        grp.assign(
            word_num_peaks=len(grp[grp.p_peak<p_peak])
        ) for i,grp in odf.groupby('word')
    )
    if min_peaks: odf=odf[odf.word_num_peaks>=min_peaks]
    return odf.sort_values('foote_novelty_z',ascending=False)

In [ ]:
get_signif_novelty_scores(
    p_peak=0.05
).groupby('word').mean().sort_values('foote_novelty_z',ascending=False)

In [ ]:
def get_signif_novelty_words(p_peak=0.05,min_peaks=1):
    df=get_all_novelty_scores()
    dfsign=get_signif_novelty_scores(p_peak=p_peak,min_peaks=min_peaks)
    signwset=set(dfsign.word)
    o=[
        w for w in 
        df.groupby('word').mean().sort_values('foote_novelty',ascending=False).index
        if w in signwset
    ]
    print('# all words',len(set(df.word)))
    print('# signif words',len(set(dfsign.word)))
    return o

In [ ]:
sign_words = get_signif_novelty_words(p_peak=0.05)
print(len(sign_words), sign_words[:5])

## Plotting

## Plotting all significant words' novelties

In [ ]:
def plot_novelty_by_foote_size(p_peak=0.01,min_peaks=1,rolling=2, ymin=-1, nudge_x=1, labsize=6,words={}):
    df=get_all_novelty_scores(by_foote_size=True, min_foote_size=4, max_foote_size=6)
    if not words: words=get_signif_novelty_words(p_peak=p_peak,min_peaks=min_peaks)
#     words={w for w in words if not 's' in w and not 'f' in w}
    print('# words used:',len(words))
    if words: df=df[df.word.isin(words)]
    figdf=pd.DataFrame([
        {
            'foote_size':fs,
            'period':period,
            'num_peaks':len(grp.query(f'p_peak<{p_peak}')),
            'avg_nov_signif':grp.query(f'p_peak<{p_peak}').foote_novelty_z.mean(),
            'avg_nov':grp.foote_novelty_z.mean(),
        } for ((fs,period),grp) in df.groupby([
            'foote_size','period'
        ])
    ])
    for ycol in ['avg_nov','avg_nov_signif']:
        figdf[ycol]=figdf[ycol].rolling(rolling,min_periods=1).mean()
    
    fig=start_fig(
        figdf,
        x='period',
        y='num_peaks',
#         size='num_peaks',
        color='factor(foote_size)',
#         linetype='factor(foote_size)',
    )
    fig+=p9.geom_line()
    fig+=p9.geom_point(p9.aes(shape='factor(foote_size)'))
    
    fig+=p9.scale_color_gray(start=.8, end=.2)
    fig+=p9.geom_vline(xintercept=1770,linetype='dotted',alpha=0.5) 
    fig+=p9.geom_vline(xintercept=1800,linetype='dotted',alpha=0.5) 
    fig+=p9.geom_vline(xintercept=1830,linetype='dotted',alpha=0.5) 
    fig+=p9.geom_label(label='Sattelzeit begins (1770)',x=1770+nudge_x,y=ymin,angle=90,size=labsize,color='black',va='bottom',boxcolor=(0,0,0,0))
    fig+=p9.geom_label(label='Sattelzeit ends (1830)',x=1830+nudge_x,y=ymin,angle=90,size=labsize,color='black',va='bottom',boxcolor=(0,0,0,0)) 
    return fig

In [ ]:
plot_novelty_by_foote_size(rolling=1, p_peak=.01, min_peaks=1)#, words={'culture'})

In [ ]:
plot_novelty_by_foote_size(rolling=1, words={'potato'})

In [ ]:
dfchangepoints=get_signif_novelty_scores(p_peak=.05, min_peaks=1).drop_duplicates('word',keep='first').sort_values('period')
dfchangepoints

In [ ]:
odfstr=pd.DataFrame([
    {'period':period, 'words':', '.join(grp.sort_values('foote_novelty_z',ascending=False).word)}
    for period,grp in sorted(dfchangepoints.groupby('period'))
])
printm(odfstr.to_markdown())

## Plotting individual words

In [ ]:
def get_plot_novelty_figdf(novdf):
    figdf=novdf.sample(frac=1)
    ywl=[
        f'{x} years'
        for x in figdf['foote_size']*5*2
    ]
    ywls=set(ywl)
    ywll=list(reversed(sorted(list(ywls))))
    figdf['year_window']=pd.Categorical(ywl, categories=ywll)
    figdf['glen']=1
    figdf['is_signif']=pd.Categorical(
        [bool(x<0.05) for x in figdf.p_peak],
        categories=[True,False]
    )
    
    figdf = pd.concat(
        grp.assign(foote_novelty_z=grp.foote_novelty.apply(lambda x: (x-grp.foote_novelty.mean())/grp.foote_novelty.std()))
        for i,grp in figdf.groupby('foote_size')
    )
    return figdf.dropna().sort_values(['year_window','period'])


# @interact
def plot_novelty(
        words=None,
        novdf=None,
        color='factor(year_window)',
        group='factor(year_window)',
        shape='factor(year_window)',
        size='glen',
        max_p_peak=None,
        vnum='v9',
        showdata=False,
        xlab='Date of semantic model',
        ylab='Foote Novelty (standardized)',
        colorlab='Foote matrix width',
        shapelab='Foote matrix width',
        sizelab='Number of significant peaks',
        title='Average novelty score for significant words over time',
        rolling=2,
        min_periods=1,
        min_foote_size=6,
        max_foote_size=6,
        y='foote_novelty',
        ymin=-.1,
        ylim0=0,
        ylim1=20,
        use_ylim=False,
        xlim0=1750,
        xlim1=1900,
        sizemin=.25,
        sizemax=2,
        labsize=6,
        hline='',
        nudge_label_y=1,
        ymin_heatmap=1750,
        combine=False,
        use_color=False,
        h_fig1=4.00,
        h_fig2=4.00,
        nudge_x=3,
        xlab_min=1735,
        add_median=True,
        save=False,
        label_words=False,
        logy=False,
        show_period_labels=True,
        dist_invert_fill=False,
        line_size=0.5,
        label_size=7,
        by_word=False
        ):

    figwords=set(words) if words else {'allwords'}
    if novdf is None:
        if words is None:
            print('neither words nor novdf')
            return
        
        novdf = get_novelty(words,by_word=by_word)
        if not by_word: words=None
        print(f'Computed novelty df of shape {novdf.shape}')
#         display(novdf)
        
#     figdf=get_plot_novelty_figdf(novdf.query(f'{min_foote_size}<=foote_size<={max_foote_size}'))
    figdf=get_plot_novelty_figdf(novdf)
    if not len(figdf): return
    if max_p_peak: figdf=figdf[figdf.p_peak<max_p_peak]
    
    
    figdf=figdf.sort_values('period')
    if showdata: display(figdf)
    fig=start_fig(
        figdf,
        x='period',
        y=y,
        color=color if color else None,
        group=group if group else None,
        figure_size=(8,h_fig1)
    )
    
    if add_median:
        kname='Guides'
        mediandf=pd.DataFrame([{
            'yintercept':figdf[y].median(),
            kname:'Median',
        },
        ])
        fig+=p9.geom_hline(
            p9.aes(yintercept='yintercept',linetype=kname),
            data=mediandf,
            size=.25,
            show_legend=True
        )
    fig+=p9.geom_line(size=line_size)
    pntd={}
    if size: pntd['size']=size
    if shape: pntd['shape']=shape
    fig+=p9.geom_point(p9.aes(**pntd))
    fig+=p9.labs(x=xlab,y=ylab,title=title,color=colorlab,size=sizelab,shape=shapelab)
    if use_ylim: fig+=p9.ylim(ylim0,ylim1)
    fig+=p9.scale_size_continuous(range=(sizemin,sizemax))
    if not use_color: fig+=p9.scale_color_gray(direction=1)# if not use_color else p9.scale_color_distiller(type='qual')
    if hline not in {None,''}:
        fig+=p9.geom_hline(yintercept=hline,linetype='dotted')
    if words and label_words:
        labeldf=figdf[figdf.is_signif==1]
        grps=[
            grp.sort_values(y).iloc[-1:]
            for i,grp in labeldf.groupby('word')
        ]
        if len(grps):
            labeldf=pd.concat(grps)
            labeldf[y]+=nudge_label_y
            fig+=p9.geom_label(p9.aes(label='word'),color='black',
                               size=label_size,data=labeldf,boxcolor=(0,0,0,0))
    if show_period_labels:
        fig+=p9.geom_vline(xintercept=1770,linetype='dotted',alpha=0.5) 
        fig+=p9.geom_vline(xintercept=1800,linetype='dotted',alpha=0.5) 
        fig+=p9.geom_vline(xintercept=1830,linetype='dotted',alpha=0.5) 
        fig+=p9.geom_label(label='Sattelzeit begins (1770)',x=1770+nudge_x,y=ymin,angle=90,size=labsize,color='black',va='bottom',boxcolor=(0,0,0,0))
        fig+=p9.geom_label(label='Sattelzeit ends (1830)',x=1830+nudge_x,y=ymin,angle=90,size=labsize,color='black',va='bottom',boxcolor=(0,0,0,0)) 
    if size=='is_signif':
        fig+=p9.scale_size_manual({True:2,False:.2})
    else:
        fig+=p9.scale_size_continuous(range=[.25,3])
    fig+=p9.theme_minimal()
    fig+=p9.theme(axis_text_x=p9.element_text(angle=90), text=p9.element_text(size=8))
    if logy: fig+=p9.scale_y_log10(limits=[ylim0,ylim1])
    fig+=p9.scale_x_continuous(
        minor_breaks=list(range(xlim0//5*5,(xlim1//5*5)+5,5)),
        limits=[xlim0,xlim1]
    )
    wkey=''
    if words: wkey=words.replace(' ','') if type(words)==str else '-'.join(words)
    ofn=f'''fig.footenov.{vnum}.{wkey+'.' if wkey else ''}{'cmbo.' if combine else ''}png'''
    ofnfn=os.path.join(PATH_FIGS,ofn)

    if combine:
        yymin1=figdf.period.min()
        yymax1=figdf.period.max()
        figdm=plot_historical_semantic_distance_matrix(words=figwords,ymin=xlim0,ymax=xlim1)
        ofig=combine_plots(figdm,fig,ofn=ofnfn)
    else:
        ofig=fig
        if save: ofig.save(ofnfn)
    display(ofig)
    if save: upfig(ofnfn)

In [ ]:
def plot_novelty_words(words,**kwargs):
    words=[w.strip() for w in words.split(',')] if type(words)==str else list(words)
    inpd=dict(
        y='foote_novelty_z',
        words=words,
        color='word',
        group='word',
        shape='word',
        colorlab='Word',
        shapelab='Word',
        sizelab='Statistically significant',
        title='Novelty scores for key words',
        ylab='Foote Novelty score',
        size='is_signif',
        vnum='v19',
        use_ylim=False,
        add_median=True,
        max_p_peak=0.0,
        min_foote_size=5,
        max_foote_size=5,
        showdata=False,
        nudge_x=2,
        logy=False,
        ylim0=0,
        ylim1=10,
        xlim0=1740,
        xlim1=1940,
        rolling=2,
        ymin=.1,
        label_words=True,
        show_period_labels=True,
        nudge_label_y=0.25,
        save=True,
        by_word=True
    )
    return plot_novelty(**{**inpd, **kwargs})


In [ ]:
# plot_novelty_words('station,value,commerce,growth,culture,slave,slavery,god,time,december')
plot_novelty_words('station,value,slave,demand,interest,circulation,improvement')